# Change long names
This notebook is designed to modify the `long_name` attribute of the variables, using the `altLabel` from the P35 Vocabulary.
In addition the `long_name` is modified so that the variable name appears first.

## Examples
```bash
Water body dissolved inorganic nitrogen -> Inorganic nitrogen_dissolved 
Deepest depth for Water body dissolved oxygen concentration → Oxygen deepest depth
```

## Input
The notebook will loop over the netCDF files located inside the directory `datadir`.

<div class="alert alert-block alert-info">
<b>Info:</b> If you need to modify the variable names, use the script <code>./src/bash/change_varnames.bash</code>.
</div>

In [70]:
using Pkg
Pkg.activate("../")
Pkg.instantiate()
using NCDatasets
using Glob
using DIVAnd
include("../emodnetchemistry.jl")

  Activating project at `~/Projects/EMODnet/EMODnet-Chemistry/src/julia`


Main.EMODnetChemistry

## Generate list of files

In [83]:
#datadir = "/home/ctroupin/data/EMODnet-Chemistry/Eutrophication2024/Results/ogs04/All_European_Seas-water_body/"

# datadir = "/media/ctroupin/T7 Shield/data/EMODnet-Chemistry/Eutrophication2024/Results/By_sea_regions-water_body/Mediterranean_Sea/"
# datadir = "/media/ctroupin/T7 Shield/data/EMODnet-Chemistry/Eutrophication2024/Results/By_sea_regions-water_body/Arctic_Ocean"
# datadir = "/media/ctroupin/T7 Shield/data/EMODnet-Chemistry/Eutrophication2024/Results/By_sea_regions-water_body/Baltic_Sea/"
# datadir = "/media/ctroupin/T7 Shield/data/EMODnet-Chemistry/Eutrophication2024/Results/By_sea_regions-water_body/Northeast_Atlantic_Ocean/"
# datadir = "/media/ctroupin/T7 Shield/data/EMODnet-Chemistry/Eutrophication2024/Results/By_sea_regions-water_body/North_Sea/"
datadir = "/media/ctroupin/T7 Shield/data/EMODnet-Chemistry/Eutrophication2024/Results/By_sea_regions-water_body/"

# datadir = "/media/ctroupin/T7 Shield/data/EMODnet-Chemistry/Eutrophication2024/Results/Coastal_areas-water_body/Mediterranean_Sea_-_Po_River"
# datadir = "/media/ctroupin/T7 Shield/data/EMODnet-Chemistry/Eutrophication2024/Results/Coastal_areas-water_body/Baltic_Sea_-_Gulf_of_Riga/"
# datadir = "/media/ctroupin/T7 Shield/data/EMODnet-Chemistry/Eutrophication2024/Results/Coastal_areas-water_body/Black_Sea-_Danube_Delta"
# datadir = "/media/ctroupin/T7 Shield/data/EMODnet-Chemistry/Eutrophication2024/Results/Coastal_areas-water_body/Northeast_Atlantic_Ocean_-_Loire_River"

datafilelist = glob("*nc", datadir);
datafilelist = EMODnetChemistry.get_file_list(datadir);
@info("Found $(length(datafilelist)) files")

[ Info: No variable selected
[ Info: No season selected
[ Info: Found 27 files


In [84]:
datafilelist

27-element Vector{Any}:
 "/media/ctroupin/T7 Shield/data/" ⋯ 104 bytes ⋯ "_oxygen_concentration.4Danl.nc"
 "/media/ctroupin/T7 Shield/data/" ⋯ 83 bytes ⋯ "/Water_body_phosphate.4Danl.nc"
 "/media/ctroupin/T7 Shield/data/" ⋯ 82 bytes ⋯ "n/Water_body_silicate.4Danl.nc"
 "/media/ctroupin/T7 Shield/data/" ⋯ 85 bytes ⋯ "er_body_chlorophyll-a.4Danl.nc"
 "/media/ctroupin/T7 Shield/data/" ⋯ 100 bytes ⋯ "ed_inorganic_nitrogen.4Danl.nc"
 "/media/ctroupin/T7 Shield/data/" ⋯ 102 bytes ⋯ "_oxygen_concentration.4Danl.nc"
 "/media/ctroupin/T7 Shield/data/" ⋯ 81 bytes ⋯ "/Water_body_phosphate.4Danl.nc"
 "/media/ctroupin/T7 Shield/data/" ⋯ 80 bytes ⋯ "a/Water_body_silicate.4Danl.nc"
 "/media/ctroupin/T7 Shield/data/" ⋯ 84 bytes ⋯ "er_body_chlorophyll-a.4Danl.nc"
 "/media/ctroupin/T7 Shield/data/" ⋯ 99 bytes ⋯ "ed_inorganic_nitrogen.4Danl.nc"
 "/media/ctroupin/T7 Shield/data/" ⋯ 101 bytes ⋯ "_oxygen_concentration.4Danl.nc"
 "/media/ctroupin/T7 Shield/data/" ⋯ 80 bytes ⋯ "/Water_body_phosphate.4Danl.nc"


### Change the long_name attribute

In [65]:
@info("Working in directory $(datadir)");
for datafile in datafilelist
    @info("Working on file $(basename(datafile))")
    NCDataset(datafile, "a") do ds
        varlist = keys(ds)

        # Identify the main variable name
        mainvar_ = first(varlist)
        # mainvar_ = first(split(last(keys(ds)), "_"))
        mainvarshort = replace(mainvar_, "Water_body_" => "", "Water body " => "", "_" => " ")
        @info("Main variable: $(mainvar_), $(mainvarshort)")

        # Get the parameter URN
        parameter_keyword_urn = ds.attrib["parameter_keyword_urn"]
        @info(parameter_keyword_urn)

        # Generate alternative label using Vocab
        newlabel = Vocab.altLabel(parameter_keyword_urn)
        @info("The new label is $(newlabel)")
        @info("")

        # Loop on variables
        for variable in varlist
            if startswith(variable, mainvar_)
                @info(" ")
                @info(variable)
                    
                if endswith(variable, "relerr")
                    @debug("Relative error")
                    new_longname = "$(newlabel) relative error"
                    @info(new_longname)
                elseif endswith(variable, "relerr")
                    @debug("Relative error")
                    new_longname = "$(newlabel) relative error"
                    @info(new_longname)
                elseif endswith(variable, "deepest")
                    @debug("Deepest value")
                    new_longname = "$(newlabel) deepest values"
                    @info(new_longname)
                elseif endswith(variable, "deepest_depth")
                    @debug("Deepest depth")
                    new_longname = "$(newlabel) deepest depth"
                    @info(new_longname)
                elseif endswith(variable, "deepest_L2")
                    @debug("Deepest masked at 50%")
                    new_longname = "$(newlabel) deepest values masked using relative error threshold 0.5"
                    @info(new_longname)
                elseif endswith(variable, "deepest_L1")
                    @debug("Deepest masked at 30%")
                    new_longname = "$(newlabel) deepest values masked using relative error threshold 0.3"
                    @info(new_longname)
                elseif endswith(variable, "_L2")
                    @debug("Masked at 50%")
                    new_longname = "$(newlabel) masked using relative error threshold 0.5"
                    @info(new_longname)
                elseif endswith(variable, "_L1")
                    @debug("Masked at 30%")
                    new_longname = "$(newlabel) masked using relative error threshold 0.3"
                    @info(new_longname)
                else
                    @debug("Full variable")
                    new_longname = "$(newlabel)"
                    @info(new_longname)
                end
                
                # Perform change
                ds[variable].attrib["long_name"] = rstrip(new_longname)
            end
    
        end
    end
    @info("");
end

[ Info: Working in directory /media/ctroupin/T7 Shield/data/EMODnet-Chemistry/Eutrophication2024/Results/By_sea_regions-water_body/North_Sea/
[ Info: Working on file Water_body_chlorophyll-a.4Danl.nc
[ Info: Main variable: Water_body_chlorophyll-a, chlorophyll-a
[ Info: SDN:P35::EPC00105
[ Info: The new label is Chlorophyll-a_particulate
[ Info: 
[ Info:  
[ Info: Water_body_chlorophyll-a
[ Info: Chlorophyll-a_particulate
[ Info:  
[ Info: Water_body_chlorophyll-a_L1
[ Info: Chlorophyll-a_particulate masked using relative error threshold 0.3
[ Info:  
[ Info: Water_body_chlorophyll-a_L2
[ Info: Chlorophyll-a_particulate masked using relative error threshold 0.5
[ Info:  
[ Info: Water_body_chlorophyll-a_deepest
[ Info: Chlorophyll-a_particulate deepest values
[ Info:  
[ Info: Water_body_chlorophyll-a_deepest_L1
[ Info: Chlorophyll-a_particulate deepest values masked using relative error threshold 0.3
[ Info:  
[ Info: Water_body_chlorophyll-a_deepest_L2
[ Info: Chlorophyll-a_particula

## Modify the standard names
https://cfconventions.org/Data/cf-standard-names/current/build/cf-standard-name-table.html

In [66]:
for datafile in datafilelist
    @info("Working on file $(datafile)")
    NCDataset(datafile, "a") do ds
        varlist = keys(ds)

        # Identify the main variable name
        mainvar_ = ds.attrib["parameter_keyword"]
        mainvar_ = replace(mainvar_, " " => "_")
        # mainvar_ = first(varlist)
        # mainvar_ = first(split(last(keys(ds)), "_"))

        # Loop on variables
        for varname in varlist
            @debug("Variable name: $(varname)")
            
            # Work only on variables related to parameter (not coordinates etc)
            if startswith(varname, mainvar_)
                varattribs = ds[varname].attrib

                # Check if there is a standard name
                if haskey(varattribs, "standard_name")
                    stdname = varattribs["standard_name"]
                    @info("++++++++++++++++++++");

                    # Perform change
                    @info("$(stdname) => $(standard_names_dict[stdname])")
                    ds[varname].attrib["standard_name"] = EMODnetChemistry.standard_names_dict[stdname]
                end  
            end
        end
    end
end

[ Info: Working on file /media/ctroupin/T7 Shield/data/EMODnet-Chemistry/Eutrophication2024/Results/By_sea_regions-water_body/North_Sea/Water_body_chlorophyll-a.4Danl.nc
[ Info: ++++++++++++++++++++
[ Info: Water_body_chlorophyll-a => mass_concentration_of_chlorophyll_a_in_sea_water
[ Info: ++++++++++++++++++++
[ Info: Water_body_chlorophyll-a => mass_concentration_of_chlorophyll_a_in_sea_water
[ Info: ++++++++++++++++++++
[ Info: Water_body_chlorophyll-a => mass_concentration_of_chlorophyll_a_in_sea_water
[ Info: ++++++++++++++++++++
[ Info: Water_body_chlorophyll-a => mass_concentration_of_chlorophyll_a_in_sea_water
[ Info: ++++++++++++++++++++
[ Info: Water_body_chlorophyll-a => mass_concentration_of_chlorophyll_a_in_sea_water
[ Info: Working on file /media/ctroupin/T7 Shield/data/EMODnet-Chemistry/Eutrophication2024/Results/By_sea_regions-water_body/North_Sea/Water_body_dissolved_oxygen_concentration.4Danl.nc
[ Info: ++++++++++++++++++++
[ Info: Water_body_dissolved_oxygen_concentr

## Create the JSON files

In [77]:
for datafile in datafilelist
    @info("Working on file $(datafile)")
    NCDataset(datafile, "a") do ds
        varlist = keys(ds)

        # Identify the main variable name
        mainvar_ = ds.attrib["parameter_keyword"]
        mainvar_ = replace(mainvar_, " " => "_")
        @info(mainvar_)

        jsonfile = datafile * ".json"
        @info("Writing JSON file $(basename(jsonfile))");
        EMODnetChemistry.write_json(jsonfile, mainvar_);
        
    end
end

[ Info: Working on file /media/ctroupin/T7 Shield/data/EMODnet-Chemistry/Eutrophication2024/Results/By_sea_regions-water_body/Arctic_Ocean/Water_body_dissolved_oxygen_concentration.4Danl.nc
[ Info: Water_body_dissolved_oxygen_concentration
[ Info: Writing JSON file Water_body_dissolved_oxygen_concentration.4Danl.nc.json
[ Info: Working on file /media/ctroupin/T7 Shield/data/EMODnet-Chemistry/Eutrophication2024/Results/By_sea_regions-water_body/Arctic_Ocean/Water_body_phosphate.4Danl.nc
[ Info: Water_body_phosphate
[ Info: Writing JSON file Water_body_phosphate.4Danl.nc.json
[ Info: Working on file /media/ctroupin/T7 Shield/data/EMODnet-Chemistry/Eutrophication2024/Results/By_sea_regions-water_body/Arctic_Ocean/Water_body_silicate.4Danl.nc
[ Info: Water_body_silicate
[ Info: Writing JSON file Water_body_silicate.4Danl.nc.json
[ Info: Working on file /media/ctroupin/T7 Shield/data/EMODnet-Chemistry/Eutrophication2024/Results/By_sea_regions-water_body/Baltic_Sea/Water_body_chlorophyll-a.4

## Create list of DOIs

In [86]:
for datafile in datafilelist
ds = NCDataset(first(datafilelist))
@info(ds.attrib["doi"])
close(ds)

[ Info: 10.13120/ac099579-5e6a-4c57-b458-df45cebb36e0


closed Dataset